# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)

        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [ ]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Apache Cassandra coding portion.

#### Creating a Cluster

In [2]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [3]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity_project
    WITH REPLICATION =
    {'class':'SimpleStrategy','replication_factor':1}
""")

#### Set Keyspace

In [4]:
session.set_keyspace('udacity_project')

# II-I. Query 1

### Query the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### Create a table named song_table_by_session_id

In [5]:
query = "CREATE TABLE IF NOT EXISTS song_table_by_session_id" 
query = query + "(session_id int, item_in_session int, artist text, song text, length float, PRIMARY KEY(session_id, item_in_session))"
session.execute(query)                 

#### Insert data into the table

In [6]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_table_by_session_id (session_id, item_in_session, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify the data

In [7]:
query = "SELECT artist, song, length FROM song_table_by_session_id WHERE session_id = 338 and item_in_session = 4"
rows = session.execute(query)

cnt = 0
for row in rows:
    cnt += 1
    if cnt == 1:
        df = pd.DataFrame(data={'artist':[row.artist], 'song':[row.song], 'length':[row.length]})
    else:
        dfnew = pd.DataFrame(data={'artist':[row.artist], 'song':[row.song], 'length':[row.length]})
        df = df.append(dfnew)
df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


# II-II. Query 2

### Query the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Create a table named song_table_by_user_id.

In [8]:
query = "CREATE TABLE IF NOT EXISTS song_table_by_user_id"
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, PRIMARY KEY((user_id, session_id), item_in_session))"
session.execute(query)                 

#### Insert data into the table

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_table_by_user_id (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Do a SELECT to verify the data

In [10]:
query = "SELECT artist, song, first_name, last_name FROM song_table_by_user_id WHERE user_id = 10 and session_id = 182"
rows = session.execute(query)

cnt = 0
for row in rows:
    cnt += 1
    if cnt == 1:
        df = pd.DataFrame(data={'artist':[row.artist], 'song':[row.song], 'first_name':[row.first_name], 'last_name':[row.last_name]})
    else:
        dfnew = pd.DataFrame(data={'artist':[row.artist], 'song':[row.song], 'first_name':[row.first_name], 'last_name':[row.last_name]})
        df = df.append(dfnew)

df

,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
0,Three Drives,Greece 2000,Sylvie,Cruz
0,Sebastien Tellier,Kilometer,Sylvie,Cruz
0,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


# II-III. Query 3

### Query every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

#### Created a table named song_table_by_song.

In [11]:
query = "CREATE TABLE IF NOT EXISTS song_table_by_song"
query = query + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY((song), user_id))"
session.execute(query)                 

#### Insert data into the table

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO song_table_by_song (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify the data

In [13]:
query = "SELECT first_name, last_name FROM song_table_by_song WHERE song = 'All Hands Against His Own'"
rows = session.execute(query)
cnt = 0
for row in rows:
    cnt += 1
    if cnt == 1:
        df = pd.DataFrame(data={'first_name':[row.first_name], 'last_name':[row.last_name]})
    else:
        dfnew = pd.DataFrame(data={'first_name':[row.first_name], 'last_name':[row.last_name]})
        df = df.append(dfnew)
df

,first_name,last_name
0,Jacqueline,Lynch
0,Tegan,Levine
0,Sara,Johnson


### Drop the tables before closing out the sessions

In [14]:
query = "DROP TABLE song_table_by_session_id"
session.execute(query)

In [15]:
query2 = "DROP TABLE song_table_by_user_id"
session.execute(query2)

In [16]:
query3 = "DROP TABLE song_table_by_song"
session.execute(query3)

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()